# Simple Image → SVG Notebook
Run top to bottom. Small sample, fast.

In [ ]:
# (Optional) installs if running outside the project folder
# %pip install -r requirements.txt
# !apt-get -y install potrace
import os, io, random, subprocess
from datasets import load_dataset
from PIL import Image
import cairosvg
from transformers import AutoModelForCausalLM, AutoProcessor
import torch

BASE = "Lica-Research-Assignment-Simple"
os.makedirs(f"{BASE}/samples/gt_svg", exist_ok=True)
os.makedirs(f"{BASE}/samples/gt_png", exist_ok=True)
os.makedirs(f"{BASE}/runs/B0_svg", exist_ok=True)
os.makedirs(f"{BASE}/runs/B1_svg", exist_ok=True)
print("Folders ready ✅")


## 1) Load a tiny sample from Hugging Face (200 items, we will use 12)

In [ ]:
ds = load_dataset("starvector/svg-stack", split="train[:200]")
items = random.sample(list(ds), 12)
print("Loaded:", len(items), "samples")


## 2) Rasterize SVG → PNG so we can see them

In [ ]:
def rasterize(svg_text, size=256):
    png = cairosvg.svg2png(bytestring=svg_text.encode(), output_width=size, output_height=size)
    return Image.open(io.BytesIO(png)).convert("RGB")

for i, ex in enumerate(items):
    with open(f"{BASE}/samples/gt_svg/{i}.svg","w") as f:
        f.write(ex["Svg"])
    rasterize(ex["Svg"]).save(f"{BASE}/samples/gt_png/{i}.png")
print("Saved GT SVG+PNG ✅")


## 3) Baseline A: Potrace (classical, no ML)

In [ ]:
def png_to_pbm(png_path, pbm_path):
    img = Image.open(png_path).convert("L").point(lambda x: 255 if x>200 else 0, mode='1')
    img.save(pbm_path)

for i in range(len(items)):
    png = f"{BASE}/samples/gt_png/{i}.png"
    pbm = f"{BASE}/runs/B0_{i}.pbm"
    out_svg = f"{BASE}/runs/B0_svg/{i}.svg"
    png_to_pbm(png, pbm)
    subprocess.run(["potrace", pbm, "-s", "-o", out_svg], check=True)
print("Potrace outputs saved ✅")


## 4) Baseline B: StarVector (AI, zero-shot)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
ckpt = "starvector/starvector-1b-im2svg"
model = AutoModelForCausalLM.from_pretrained(ckpt).to(device).eval()
processor = AutoProcessor.from_pretrained(ckpt)
print("Model loaded on", device)

def im2svg(png_path):
    image = Image.open(png_path).convert("RGB")
    inputs = processor(images=image, text="Generate SVG:", return_tensors="pt").to(device)
    with torch.inference_mode():
        out = model.generate(**inputs, max_new_tokens=1024, do_sample=False)
    txt = processor.batch_decode(out, skip_special_tokens=True)[0]
    s, e = txt.find("<svg"), txt.rfind("</svg>")
    return txt[s:e+6] if s!=-1 and e!=-1 else txt

for i in range(len(items)):
    svg_text = im2svg(f"{BASE}/samples/gt_png/{i}.png")
    with open(f"{BASE}/runs/B1_svg/{i}.svg","w") as f:
        f.write(svg_text)
print("StarVector outputs saved ✅")
